# PYGVEC PERFORMANCE TEST

In [ ]:
from pathlib import Path

# replace with paths to your own data

example="frenet_axisNB_QH_nfp7" #hmap_axisNB
#example="frenet_QH_nfp7_untwisted" #hmap_frenet
#example="w7x"  #hmap_RZ
#example="ellipstell_lowres" #hmap_RZ
basepath = Path("/home/fhinde/GIT/gvec_hmap_performance/build/run/example/" + example)
parameterfile = basepath / "parameter.ini"
statefile = sorted(basepath.glob("*State*.dat"))[-1]
assert parameterfile.exists() and statefile.exists()

In [ ]:
%%writefile profiling_run.py
import os
os.environ["OMP_NUM_THREADS"] = "2"

from pathlib import Path

import numpy as np

import gvec


def doprof_eval(parameterfile,statefile,vars=(),nrho=10,ntheta=101,nzeta=101):
    with gvec.State(parameterfile, statefile) as state:
        rho = np.linspace(1e-4,1,nrho)
        ev = gvec.Evaluations(rho, ntheta, nzeta, state=state)
        state.compute(ev, *vars)
    return ev

def doprof_eval_boozer(parameterfile,statefile,vars=(),nrho=10,ntheta=101,nzeta=101):
    with gvec.State(parameterfile, statefile) as state:
        rho = np.linspace(1e-4,1,nrho)
        ev = gvec.EvaluationsBoozer(rho, ntheta, nzeta, state=state)
        state.compute(ev, *vars)
    return ev

In [ ]:
from profiling_run import doprof_eval
print(f"running example {example}")
%prun -l 10 ev=doprof_eval(parameterfile,statefile,vars=("B","pos","I_tor","I_pol",))

In [ ]:
from profiling_run import doprof_eval 
print(f"running example {example}")
%prun -l 10 ev=doprof_eval(parameterfile,statefile,vars=("J",))

In [ ]:
ev.to_netcdf(f"ev_compute_{example}.nc")

In [ ]:
from profiling_run import doprof_eval_boozer
print(f"running example {example}")
%prun -l 10 ev=doprof_eval_boozer(parameterfile,statefile,vars=("B","pos",))

In [ ]:
ev.to_netcdf(f"ev_compute_{example}_boozer.nc")